<div style="padding: 10px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden)"><b><span style='color:Yellow;font-size:60px; font-family:Times New Roman;'> GOLD PREDICTION </span></b> </div>

 <h1 style="border:3px solid darkred; padding: 15px;color:white;margin:10;font-size:200%;text-align:center;display:fill;border-radius:10px;overflow:hidden; color:darkred;font-weight:bold; font-size:45px;">Importing Libraries</h1>